In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd '/content/drive/MyDrive/Computer_Vision_Final_Project/Computer_Vision_Final_Dataset'

/content/drive/.shortcut-targets-by-id/1qRPzc_kkTeNinCAOMoXj6d_nwu-u_HfH/Computer_Vision_Final_Project/Computer_Vision_Final_Dataset


In [3]:
from __future__ import print_function, division
import cv2
import os
import torch
import pandas as pd
from skimage import io, transform
from skimage.color import rgba2rgb
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

from albumentations.pytorch.functional import img_to_tensor

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

plt.ion()   # interactive mode

#**Helper Functions**

In [4]:
class Rescale(object):
    """Rescale the image in a sample to a given size.

    Args:
        output_size (tuple or int): Desired output size. If tuple, output is
            matched to output_size. If int, smaller of image edges is matched
            to output_size keeping aspect ratio the same.
    """

    def __init__(self, output_size):
        assert isinstance(output_size, (int, tuple))
        self.output_size = output_size

    def __call__(self, sample):
        # image
        image = sample['image']

        h, w = image.shape[:2]
        if isinstance(self.output_size, int):
            if h > w:
                new_h, new_w = self.output_size * h / w, self.output_size
            else:
                new_h, new_w = self.output_size, self.output_size * w / h
        else:
            new_h, new_w = self.output_size

        new_h, new_w = int(new_h), int(new_w)

        img = transform.resize(image, (new_h, new_w))

        # label
        label = sample['ground_truth']

        h, w = label.shape[:2]
        if isinstance(self.output_size, int):
            if h > w:
                new_h, new_w = self.output_size * h / w, self.output_size
            else:
                new_h, new_w = self.output_size, self.output_size * w / h
        else:
            new_h, new_w = self.output_size

        new_h, new_w = int(new_h), int(new_w)

        label = transform.resize(label, (new_h, new_w))

        return {'image': img, 'ground_truth': label}


class RandomCrop(object):
    """Crop randomly the image in a sample.

    Args:
        output_size (tuple or int): Desired output size. If int, square crop
            is made.
    """

    def __init__(self, output_size):
        assert isinstance(output_size, (int, tuple))
        if isinstance(output_size, int):
            self.output_size = (output_size, output_size)
        else:
            assert len(output_size) == 2
            self.output_size = output_size

    def __call__(self, sample):
        # image
        image  = sample['image']

        h, w = image.shape[:2]
        new_h, new_w = self.output_size

        top = np.random.randint(0, h - new_h)
        left = np.random.randint(0, w - new_w)

        image = image[top: top + new_h,
                      left: left + new_w]

        # label
        label  = sample['ground_truth']

        h, w = label.shape[:2]
        new_h, new_w = self.output_size

        top = np.random.randint(0, h - new_h)
        left = np.random.randint(0, w - new_w)

        label = label[top: top + new_h,
                      left: left + new_w]

        return {'image': image, 'ground_truth': label}

class Normalize(object):
    """Normalize images"""

    def __call_(self, sample):
        # image
        image = sample['image']
        normalize = {"mean": [0.485, 0.456, 0.406],
                    "std": [0.229, 0.224, 0.225]}
        image = img_to_tensor(image[0], normalize).unsqueeze(0)

        # label
        label = sample['ground_truth']
        normalize = {"mean": [0.485, 0.456, 0.406],
                    "std": [0.229, 0.224, 0.225]}
        label = img_to_tensor(label[0], normalize).unsqueeze(0)

        return {'image': image, 'ground_truth': label}

class ToTensor(object):
    """Convert ndarrays in sample to Tensors."""

    def __call__(self, sample):
        # image
        image = sample['image']

        # swap color axis because
        # numpy image: H x W x C
        # torch image: C x H x W
        image = image.transpose((2, 0, 1))

        # label
        label = sample['ground_truth']

        # swap color axis because
        # numpy label: H x W x C
        # torch label: C x H x W
        label = label.transpose((2, 0, 1))

        return {'image': torch.from_numpy(image), 'ground_truth': torch.from_numpy(label)}

def rgb2gray(rgb):
    b, g, r = rgb[:, :, 0], rgb[:, :, 1], rgb[:, :, 2]
    gray = 0.2989*r + 0.5870*g + 0.1140*b
    gray = np.expand_dims(gray, 2)
    return gray

#**Copy&Paste**


In [5]:
class CopyPasteDataset(Dataset):
    """Photoshop Detection dataset."""

    def __init__(self, csv_file, root_dir, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.copypaste_dataset = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.copypaste_dataset)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        # image
        img_path = os.path.join(self.root_dir,
                                self.copypaste_dataset.iloc[idx, 0])
        image = io.imread(img_path)
        h, w = image.shape[:2]
        image = cv2.resize(image, (512, 512))
        if image.shape[2] == 4:
          image = rgba2rgb(image)

        #  ground truth
        label_name = os.path.join(self.root_dir,
                                self.copypaste_dataset.iloc[idx, 1])
        label = io.imread(label_name)
        transforms.CenterCrop((h,w))
        label = cv2.resize(label, (512, 512))
        if label.shape[2] == 4:
          label = rgba2rgb(label)
        label = rgb2gray(label)

        sample = {'image': image, 'ground_truth': label }

        if self.transform:
            sample = self.transform(sample)

        # You need to return as tuple so later you can iterate as (for inputs, targets in dataloader)
        return sample['image'], sample['ground_truth']

In [6]:
copypaste_dataset = CopyPasteDataset(csv_file='PhotoshopDetectionDataset.csv', 
                                     root_dir='',
                                     transform=transforms.Compose([
                                               Rescale(512),
                                               RandomCrop(224),
                                               ToTensor(),
                                    ]))

In [7]:
%cd ..

/content/drive/.shortcut-targets-by-id/1qRPzc_kkTeNinCAOMoXj6d_nwu-u_HfH/Computer_Vision_Final_Project
